In [31]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, classification_report, f1_score
import time
from datetime import datetime

In [32]:
model_name = 'Naive Model'

In [33]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(y_df["is_changed_trend_test"][:-1], y_df["is_changed_trend_predict"][:-1], digits=4)

In [34]:
train_set = pd.read_csv("../../../data/post_cleaning/training_set.csv")
validation_set = pd.read_csv("../../../data/post_cleaning/validation_set.csv")

In [35]:
y_val = validation_set['target_trend']
y_val_pred = validation_set['trend_d0']

In [36]:
report = classification_report(y_val, y_val_pred, digits=4)
print(f"NaiveModel report:\n {report}")
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_val_pred))

opt_duration_sec = 0  # No hubo optimización en el modelo ingenuo

n_trials_run = 0  # No hubo trials en el modelo ingenuo

NaiveModel report:
               precision    recall  f1-score   support

          -1     0.8242    0.8152    0.8197        92
           0     0.3469    0.3469    0.3469        49
           1     0.8312    0.8366    0.8339       153

    accuracy                         0.7483       294
   macro avg     0.6674    0.6663    0.6668       294
weighted avg     0.7483    0.7483    0.7483       294

Balanced accuracy: 0.6662524913346982


In [37]:
print(trend_changes_score(y_val, y_val_pred))

              precision    recall  f1-score   support

       False     0.8219    0.8219    0.8219       219
        True     0.4730    0.4730    0.4730        74

    accuracy                         0.7338       293
   macro avg     0.6474    0.6474    0.6474       293
weighted avg     0.7338    0.7338    0.7338       293



In [38]:
# --- Exportar y Comparar Métricas de Modelos (Validación) ---

# 1. Definir el nombre del modelo actual y el archivo de salida
output_file = '../../../score_models/model_comparison_metrics.csv'

# 2. Calcular el reporte de clasificación estándar
report_dict = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score = report_dict['macro avg']['f1-score']


# 3. Calcular el reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar las nuevas métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score,
    "optuna_duration_sec": opt_duration_sec,
    "n_trials": n_trials_run,
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
try:
    # Intentar cargar el archivo existente
    comparison_df = pd.read_csv(output_file, index_col='model')
    # Si existe, actualizar o añadir la fila para el modelo actual
    comparison_df.loc[model_name] = new_metrics
except FileNotFoundError:
    # Si no existe, crear un DataFrame nuevo directamente con los datos actuales
    comparison_df = pd.DataFrame([new_metrics], index=[model_name])

# Guardar el DataFrame actualizado en el CSV
comparison_df.to_csv(output_file, index_label='model')